In [1]:
# ! pip install transformers
import torch
import torch.nn.functional as F
import numpy as np
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW
from torch.optim import Adam

import torchsummary as summary
from tqdm import tqdm

from torch import nn
from transformers import BertModel
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
import torch

import pandas as pd
import numpy as np
import os
import base64
import gc

C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def create_onedrive_directdownload(onedrive_link):
    data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
    data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
    return resultUrl

In [3]:
onedrive_link='https://1drv.ms/u/s!AoiE7xOoBAsngsgmC-PI9ray-gfELQ?e=UtffII'
onedrive_direct_link=create_onedrive_directdownload(onedrive_link)
onedrive_direct_link
df=pd.read_csv(onedrive_direct_link)
df.drop('Unnamed: 0',axis=1,inplace=True)
df=df[['news_content','rule_labels_comb']]
df=df[~df['rule_labels_comb'].isna()]
df=df[(df['rule_labels_comb'].apply(len)!=2)]
df = df.reset_index()
del df['index']
df

,news_content,rule_labels_comb
0,The article discusses how M.M. Warburg may hav...,['legal']
1,The Cologne Prosecutor's Office accuses German...,"['collaboration', 'legal']"
2,Deutsche Bank AG has acquired an additional 49...,['collaboration']
3,Mozambique has reached a negotiated agreement ...,"['collaboration', 'legal']"
4,The UBS APAC Sustainable Finance Conference 20...,"['personnel', 'legal']"
...,...,...
2589,Arbeiten bei HSBC Trinkaus & Burkhardt GmbH in...,['product']
2590,HSBC muss MillionenStrafe wegen WhatsAppNutzun...,['personnel']
2591,Alle offenen Sozialkompetent Jobs bei Bethmann...,['personnel']
2592,Citigroup Inc. Deutsche Bank AG HSBC Holdings ...,['legal']


In [4]:
from ast import literal_eval
df['rule_labels_comb'] = df['rule_labels_comb'].apply(literal_eval)
df['target']='[0.0,0.0,0.0,0.0]'
df['target'] = df['target'].apply(literal_eval)

df['target'].iloc[0][0]

0.0

In [5]:
for index, value in df['rule_labels_comb'].iteritems():
    class_dic={'personnel':0,'product':1,'collaboration':2,'legal':3}
    val=[class_dic.get(i, i) for i in value]
    df['rule_labels_comb'].iloc[index]=val
    y=0
    for x in df['rule_labels_comb'].iloc[index]:
        df['target'].iloc[index][x]=1.0
        y=y+1
df['rule_labels_comb']

0          [3]
1       [2, 3]
2          [2]
3       [2, 3]
4       [0, 3]
         ...  
2589       [1]
2590       [0]
2591       [0]
2592       [3]
2593       [3]
Name: rule_labels_comb, Length: 2594, dtype: object

In [6]:
df['target']

0       [0.0, 0.0, 0.0, 1.0]
1       [0.0, 0.0, 1.0, 1.0]
2       [0.0, 0.0, 1.0, 0.0]
3       [0.0, 0.0, 1.0, 1.0]
4       [1.0, 0.0, 0.0, 1.0]
                ...         
2589    [0.0, 1.0, 0.0, 0.0]
2590    [1.0, 0.0, 0.0, 0.0]
2591    [1.0, 0.0, 0.0, 0.0]
2592    [0.0, 0.0, 0.0, 1.0]
2593    [0.0, 0.0, 0.0, 1.0]
Name: target, Length: 2594, dtype: object

In [7]:
# import data from gdrive
'''
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/')


df=pd.read_csv('BERT_data.csv')
df=df[~(df['content']=='nan')]
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nos.chdir('/content/drive/My Drive/')\n\n\ndf=pd.read_csv('BERT_data.csv')\ndf=df[~(df['content']=='nan')]\ndf['content']=df['content'].astype(str)\ndf['subject']=df['subject'].astype(str)\n"

In [8]:
# change range of labels, minimum should be zero
df['rule_labels_comb']=df['rule_labels_comb'].astype(str)
# df['target']=df['target'].apply(lambda x: list(np.array(x) / sum(x)))
df

,news_content,rule_labels_comb,target
0,The article discusses how M.M. Warburg may hav...,[3],"[0.0, 0.0, 0.0, 1.0]"
1,The Cologne Prosecutor's Office accuses German...,"[2, 3]","[0.0, 0.0, 1.0, 1.0]"
2,Deutsche Bank AG has acquired an additional 49...,[2],"[0.0, 0.0, 1.0, 0.0]"
3,Mozambique has reached a negotiated agreement ...,"[2, 3]","[0.0, 0.0, 1.0, 1.0]"
4,The UBS APAC Sustainable Finance Conference 20...,"[0, 3]","[1.0, 0.0, 0.0, 1.0]"
...,...,...,...
2589,Arbeiten bei HSBC Trinkaus & Burkhardt GmbH in...,[1],"[0.0, 1.0, 0.0, 0.0]"
2590,HSBC muss MillionenStrafe wegen WhatsAppNutzun...,[0],"[1.0, 0.0, 0.0, 0.0]"
2591,Alle offenen Sozialkompetent Jobs bei Bethmann...,[0],"[1.0, 0.0, 0.0, 0.0]"
2592,Citigroup Inc. Deutsche Bank AG HSBC Holdings ...,[3],"[0.0, 0.0, 0.0, 1.0]"


In [9]:
# additional filtering to balance classes
'''
df_3=df[(df['result']==3) & (df['content'].str.len()<350)]
df=df[~(df['result']==3)]
df=pd.concat([df,df_3])
df
'''

"\ndf_3=df[(df['result']==3) & (df['content'].str.len()<350)]\ndf=df[~(df['result']==3)]\ndf=pd.concat([df,df_3])\ndf\n"

In [10]:
# import BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.labels=df['target']
        self.text=[tokenizer(text,padding='max_length',truncation=True,return_tensors="pt") for text in df['news_content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [11]:
# train test split
X_train, X_test= train_test_split(df[['news_content','target']], test_size=0.25,stratify=df['target'])

In [12]:
df_train=X_train
df_val=X_test
df_test=0

In [13]:
df_train

,news_content,target
415,"According to Berenberg Bank's Chief Economist,...","[0.0, 1.0, 0.0, 0.0]"
739,"Oddo BHF, a financial services company, has ma...","[0.0, 1.0, 0.0, 0.0]"
630,"Oddo BHF, a German financial services company,...","[0.0, 1.0, 0.0, 0.0]"
2335,Major Wall Street banks must face a lawsuit ac...,"[0.0, 0.0, 1.0, 1.0]"
1712,German banking company Commerzbank is set to r...,"[1.0, 1.0, 0.0, 0.0]"
...,...,...
1709,Analysts at Deutsche Bank have raised their pr...,"[1.0, 0.0, 0.0, 1.0]"
1536,"According to a report from finews.ch, Wafik Be...","[0.0, 1.0, 1.0, 1.0]"
2262,Two senior investment bankers at Credit Suisse...,"[0.0, 0.0, 1.0, 0.0]"
2304,The DAX had a sluggish start on Fronleichnam h...,"[0.0, 0.0, 1.0, 0.0]"


In [14]:
# BERT classifier architecture, with 7 output classes
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 4)
        torch.nn.init.kaiming_uniform_(self.linear.weight, nonlinearity='relu')
        self.relu = nn.ReLU()
        
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


In [15]:
# change runtype to GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [16]:
# hyperparameters
EPOCHS = 22
model = BertClassifier()
LR = 0.00006

In [17]:
df_train=df_train.reset_index()
df_train.drop('index',axis=1,inplace=True)

df_val=df_val.reset_index()
df_val.drop('index',axis=1,inplace=True)
df_val

,news_content,target
0,"<br>1. German companies, including KGaA Helm A...","[1.0, 1.0, 1.0, 1.0]"
1,In dem Artikel wird Ã¼ber den Prozess gegen Ch...,"[0.0, 1.0, 1.0, 1.0]"
2,The article is about regulatory filings made b...,"[0.0, 0.0, 1.0, 0.0]"
3,The article discusses a court showdown between...,"[0.0, 0.0, 1.0, 1.0]"
4,German-based Berenberg Bank has set a target p...,"[0.0, 1.0, 0.0, 0.0]"
...,...,...
644,Fitch Ratings has affirmed the BBB long-term i...,"[0.0, 0.0, 1.0, 0.0]"
645,"UBS, a Swiss investment bank, is applying its ...","[0.0, 1.0, 0.0, 0.0]"
646,This German article discusses the downward tre...,"[1.0, 0.0, 0.0, 1.0]"
647,The article discusses how Swiss cantonal banks...,"[0.0, 0.0, 1.0, 0.0]"


In [18]:
# display BERT layers
n=0
for x in model.state_dict():
    n=n+1
    print(x)
n

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

201

In [19]:
# freeze first 8 layers 
n=0
for param in model.parameters():
    n=n+1
    param.requires_grad = False
    if n==(201-68):
        break

In [20]:
# change datatypes of input data
df_train['news_content']=df_train['news_content'].astype(str)

df_val['news_content']=df_val['news_content'].astype(str)
df_train['news_content'].iloc[0]

"According to Berenberg Bank's Chief Economist, Holger Schmieding, global trade is expected to become less globalized in the future, shifting towards regionalization. This transformation will involve a shift from goods trade to services trade, which could benefit countries like the United States and India, who specialize in services."

In [22]:
torch.cuda.empty_cache()
gc.collect()

0

In [23]:
import copy
def accuracy_calc(l):
    p=l.tolist()
    out=[]
    for a in p:
        
        b={}
        for i,x in enumerate(a):
            b[i]=x
        sorted_dict = dict(sorted(b.items(), key=lambda item: item[1], reverse=True))
        copy_dic=sorted_dict
        copy_dic = copy.deepcopy(sorted_dict)
        su=0
        for p,q in sorted_dict.items():
            su=su+q
            copy_dic[p]=1
            if su>sum(list(sorted_dict.values()))*.8:
                break
        l=list(dict(sorted(copy_dic.items(), key=lambda item: item[0])).values())
        out.append([0 if x != 1 else x for x in l])    
    return out

In [24]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)
    
    # mini batching
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=30)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=30)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)
    
    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            n=0
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device) # to cuda GPU
                mask = train_input['attention_mask'].to(device) # attention mask
                input_id = train_input['input_ids'].squeeze(1).to(device)
                
                l1_loss=0
                
                # for L1 regularization
                a=0
                reg_loss = 0
                for param in model.parameters():
                    a=a+1
                    if a >=201-68:
                        reg_loss += torch.norm(param, 1) 
                '''
                factor = 0.00001 #lambda for L1 regularization
                l1_loss=factor * reg_loss # L1 loss
                '''
                # model output
                output = model(input_id, mask)
                
                
                # loss value
                batch_loss = criterion(output, train_label) + l1_loss
                total_loss_train += batch_loss.item() 
                
                # train accuracy 
                output=output.squeeze(0)
                train_label=train_label.squeeze(0)
                # print(output)
                # print(train_label)
                # print(torch.gather(train_label, 1, torch.argmax(output,dim=1).view(-1, 1)))
                acc = torch.gather(train_label, 1, torch.argmax(output,dim=1).view(-1, 1)).sum().item()
                total_acc_train += acc
                
                # backpropogation
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0
            
            # for validation accuracy
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)
                    
                    # validation output
                    output = model(input_id, mask)
                    
                    # validation loss value
                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    
                    # validation accuracy
                    output=output.squeeze(0)
                    val_label=val_label.squeeze(0)
                    '''
                    acc = torch.gather(val_label, 1, torch.argmax(output,dim=1).view(-1, 1)).sum().item()
                    
                    '''
                    acc_bool=(torch.tensor(accuracy_calc(output)).to(device)==val_label).int()
                    acc_tensor = torch.all(acc_bool == 1, dim=1)
                    acc=sum(acc_tensor)
                    total_acc_val += acc
                '''    
                mean = torch.mean(output, dim=1)
                std = torch.std(output, dim=1)
                output = (output - torch.unsqueeze(mean,0).t()) / torch.unsqueeze(std,0).t()
                '''
                
                print(output)
                print(val_label)

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')



train(model, df_train, df_val, LR, EPOCHS)

100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [01:58<00:00,  1.83s/it]


Epochs: 1 | Train Loss:  0.074                 | Train Accuracy:  0.430                 | Val Loss:  0.074                 | Val Accuracy:  0.159


100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [01:58<00:00,  1.82s/it]


Epochs: 2 | Train Loss:  0.071                 | Train Accuracy:  0.545                 | Val Loss:  0.070                 | Val Accuracy:  0.299


100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [01:58<00:00,  1.82s/it]


...

Epochs: 4 | Train Loss:  0.055                 | Train Accuracy:  0.865                 | Val Loss:  0.055                 | Val Accuracy:  0.450


100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [02:02<00:00,  1.88s/it]


Epochs: 5 | Train Loss:  0.048                 | Train Accuracy:  0.925                 | Val Loss:  0.055                 | Val Accuracy:  0.368


 11%|████████▉                                                                          | 7/65 [00:13<01:52,  1.95s/it]


KeyboardInterrupt: 

In [108]:
import copy
def accuracy_calc(l):
    p=l.tolist()
    out=[]
    for a in p:
        
        b={}
        for i,x in enumerate(a):
            b[i]=x
        sorted_dict = dict(sorted(b.items(), key=lambda item: item[1], reverse=True))
        copy_dic=sorted_dict
        copy_dic = copy.deepcopy(sorted_dict)
        su=0
        for p,q in sorted_dict.items():
            su=su+q
            copy_dic[p]=1
            if su>sum(list(sorted_dict.values()))*.9:
                break
        l=list(dict(sorted(copy_dic.items(), key=lambda item: item[0])).values())
        out.append([0 if x != 1 else x for x in l])    
    return out

In [109]:
import torch
z=torch.tensor([[1.0858, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1444, 2.4341, 0.0000],
        [0.0000, 1.8987, 0.0488, 0.0000],
        [0.0000, 2.2600, 1.4062, 0.0000],
        [0.0000, 2.0379, 0.8080, 0.0000],
        [0.0000, 0.0000, 1.0410, 0.0000],
        [0.0000, 1.8934, 0.1403, 0.0000],
        [0.0000, 0.0000, 1.9777, 0.0000],
        [0.0000, 3.9491, 1.2324, 0.0000],
        [0.0000, 0.9085, 1.7385, 0.0000],
        [0.0000, 0.1752, 0.0000, 0.0000],
        [0.0000, 0.0223, 2.1809, 0.0000],
        [0.0000, 2.0517, 0.1653, 0.0000],
        [0.0000, 3.8410, 1.6498, 0.0000],
        [0.0000, 0.5239, 3.3225, 0.0000],
        [0.0000, 0.6871, 0.4947, 0.0000],
        [0.0000, 2.7512, 0.4298, 0.0000],
        [0.0000, 0.3287, 0.7976, 0.2881],
        [0.0710, 0.7761, 1.7600, 0.0000]])
z

tensor([[1.0858, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1444, 2.4341, 0.0000],
        [0.0000, 1.8987, 0.0488, 0.0000],
        [0.0000, 2.2600, 1.4062, 0.0000],
        [0.0000, 2.0379, 0.8080, 0.0000],
        [0.0000, 0.0000, 1.0410, 0.0000],
        [0.0000, 1.8934, 0.1403, 0.0000],
        [0.0000, 0.0000, 1.9777, 0.0000],
        [0.0000, 3.9491, 1.2324, 0.0000],
        [0.0000, 0.9085, 1.7385, 0.0000],
        [0.0000, 0.1752, 0.0000, 0.0000],
        [0.0000, 0.0223, 2.1809, 0.0000],
        [0.0000, 2.0517, 0.1653, 0.0000],
        [0.0000, 3.8410, 1.6498, 0.0000],
        [0.0000, 0.5239, 3.3225, 0.0000],
        [0.0000, 0.6871, 0.4947, 0.0000],
        [0.0000, 2.7512, 0.4298, 0.0000],
        [0.0000, 0.3287, 0.7976, 0.2881],
        [0.0710, 0.7761, 1.7600, 0.0000]])

In [116]:
res=(torch.tensor(accuracy_calc(z)).to(device)==torch.tensor([[1., 0., 0., 0.],
        [0., 0., 1., 1.],
        [0., 1., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 1., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 1., 0.]]).to(device)).int()

In [117]:
result = torch.all(res == 1, dim=1)

# Convert the boolean tensor to integers (1 for True, 0 for False)
result = result.int()

print(result)

tensor([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       device='cuda:0', dtype=torch.int32)


In [119]:
sum(result)/len(result)

tensor(0.2105, device='cuda:0')